In [9]:
from bs4 import BeautifulSoup
import requests
from datetime import datetime

In [15]:
dict = {
'marka': 'audi',
'model':'a3 a3 hatchback',
'il': 'istanbul',
'vites':'manuel',
'yil':{
    'max':'2023',
    'min': '2000'
},
'price':{
    'min':'1',
    'max':'10000000'
}

}
root = 'https://www.sahibinden.com/'

headers = {
        'User-Agent':
        'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1)' +
        'AppleWebKit/537.36 (KHTML, like Gecko)' +
        'Chrome/39.0.2171.95 Safari/537.36',
        'Upgrade-Insecure-Requests': '1'}


In [12]:

    modelURL = (root + dict['marka'] + '-' + dict['model']).replace(' ','-')
    yilquery = 'a5min=' + dict['yil']['min'] + '&' +  'a5max=' + dict['yil']['max']
    pricequery= 'price_min=' + dict['price']['min'] + '&' + 'price_max=' + dict['price']['max']
    queries = '?'+yilquery+pricequery
    pages = modelURL + '/'+dict['il']+'/'+dict['vites']
    URL = pages + queries
    

In [13]:
URL

'https://www.sahibinden.com/audi-a3-a3-hatchback/istanbul/manuel?a5min=2000&a5max=2023price_min=1&price_max=10000000'

In [16]:
r = requests.get(URL, headers=headers)
soup = BeautifulSoup(r.text, 'html.parser')
print('response: ' + str(r.status_code))


response: 200


In [17]:
sayfa_sayisi = int(soup.find('div',{'class':'mtmdef pageNavigator pvdef phdef'}).get_text().split()[1])
print(sayfa_sayisi)

3


In [18]:

ilanlar = []
for offset in range(0, sayfa_sayisi*20, 20):
  try:
    offset = '&pagingOffset='+str(offset)
    URL_ = URL + offset
    # start scraping the page
    r = requests.get(URL_, headers=headers)
    print(r.status_code, URL_)
    soup = BeautifulSoup(r.text, 'html.parser')
    for i in soup.find_all('tr', {'class':'searchResultsItem'}):
        temp=[]
        if i.find('td', {'class':'searchResultsLocationValue'}):
            temp.append(i.find('td', {'class':'searchResultsLocationValue'}).get_text(separator = ' ').replace('\n','').replace('\r', '').replace('\t', '').strip())
        if i.find('td',{'class':'searchResultsTagAttributeValue'}):
            temp.append(i.find('td', {'class':'searchResultsTagAttributeValue'}).get_text(separator = ' ').replace('\n','').replace('\r', '').replace('\t', '').strip())
        if i.find('td',{'class':'searchResultsAttributeValue'}):
            for j in i.find_all('td',{'class':'searchResultsAttributeValue'}):
                temp.append(j.get_text(separator = ' ').replace('\n','').replace('\r', '').replace('\t', '').strip())
        if len(temp) > 0:
            ilanlar.append(temp)
  except Exception as e:
    print("Exception occured: " + str(e))
    continue

200 https://www.sahibinden.com/audi-a3-a3-hatchback/istanbul/manuel?a5min=2000&a5max=2023price_min=1&price_max=10000000&pagingOffset=0
200 https://www.sahibinden.com/audi-a3-a3-hatchback/istanbul/manuel?a5min=2000&a5max=2023price_min=1&price_max=10000000&pagingOffset=20
200 https://www.sahibinden.com/audi-a3-a3-hatchback/istanbul/manuel?a5min=2000&a5max=2023price_min=1&price_max=10000000&pagingOffset=40


In [19]:
len(ilanlar)

45

In [20]:
ilanlar

[['Silivri', 'A3 Hatchback 2.0 TDI', '2004', '220.000', 'Füme'],
 ['Pendik Çarşı', 'A3 Hatchback 1.6', '2004', '300.000', 'Kırmızı'],
 ['Tuzla Tersaneler', 'A3 Hatchback 1.6', '2004', '235.000', 'Kırmızı'],
 ['Güngören', 'A3 Hatchback 1.6 FSI', '2007', '202.000', 'Siyah'],
 ['Büyükçekmece', 'A3 Hatchback 1.6', '2004', '333.500', 'Kırmızı'],
 ['Beylikdüzü Kavaklı', 'A3 Hatchback 1.4 TFSI', '2009', '270.000', 'Gri'],
 ['Bağcılar', 'A3 Hatchback 1.6', '2006', '320.000', 'Siyah'],
 ['Sultangazi Cebeci', 'A3 Hatchback 1.6', '2001', '266.000', 'Siyah'],
 ['Kağıthane', 'A3 Hatchback 1.4 TFSI', '2008', '86.000', 'Siyah'],
 ['Esenyurt', 'A3 Hatchback 1.6', '2004', '373.000', 'Gri'],
 ['Kartal Yakacık', 'A3 Hatchback 1.6', '2006', '271.000', 'Siyah'],
 ['Sultanbeyli Turgutreis', 'A3 Hatchback 1.6', '2004', '195.000', 'Mavi'],
 ['Ataşehir Barbaros', 'A3 Hatchback 1.6 FSI', '2006', '266.500', 'Beyaz'],
 ['Pendik Çamçeşme', 'A3 Hatchback 1.6 FSI', '2007', '202.000', 'Siyah'],
 ['Üsküdar Bulgurlu', 

In [22]:
import pandas as pd
 
 
# Create the pandas DataFrame
df = pd.DataFrame(ilanlar, columns = ['konum', 'model','model_yil','km','renk'])

In [24]:
df

,konum,model,model_yil,km,renk
0,Silivri,A3 Hatchback 2.0 TDI,2004,220.000,Füme
1,Pendik Çarşı,A3 Hatchback 1.6,2004,300.000,Kırmızı
2,Tuzla Tersaneler,A3 Hatchback 1.6,2004,235.000,Kırmızı
3,Güngören,A3 Hatchback 1.6 FSI,2007,202.000,Siyah
4,Büyükçekmece,A3 Hatchback 1.6,2004,333.500,Kırmızı
5,Beylikdüzü Kavaklı,A3 Hatchback 1.4 TFSI,2009,270.000,Gri
6,Bağcılar,A3 Hatchback 1.6,2006,320.000,Siyah
7,Sultangazi Cebeci,A3 Hatchback 1.6,2001,266.000,Siyah
8,Kağıthane,A3 Hatchback 1.4 TFSI,2008,86.000,Siyah
9,Esenyurt,A3 Hatchback 1.6,2004,373.000,Gri


In [33]:
out = df.to_json(orient='records',force_ascii=False)

In [34]:
out

'[{"konum":"Silivri","model":"A3 Hatchback 2.0 TDI","model_yil":"2004","km":"220.000","renk":"Füme"},{"konum":"Pendik Çarşı","model":"A3 Hatchback 1.6","model_yil":"2004","km":"300.000","renk":"Kırmızı"},{"konum":"Tuzla Tersaneler","model":"A3 Hatchback 1.6","model_yil":"2004","km":"235.000","renk":"Kırmızı"},{"konum":"Güngören","model":"A3 Hatchback 1.6 FSI","model_yil":"2007","km":"202.000","renk":"Siyah"},{"konum":"Büyükçekmece","model":"A3 Hatchback 1.6","model_yil":"2004","km":"333.500","renk":"Kırmızı"},{"konum":"Beylikdüzü Kavaklı","model":"A3 Hatchback 1.4 TFSI","model_yil":"2009","km":"270.000","renk":"Gri"},{"konum":"Bağcılar","model":"A3 Hatchback 1.6","model_yil":"2006","km":"320.000","renk":"Siyah"},{"konum":"Sultangazi Cebeci","model":"A3 Hatchback 1.6","model_yil":"2001","km":"266.000","renk":"Siyah"},{"konum":"Kağıthane","model":"A3 Hatchback 1.4 TFSI","model_yil":"2008","km":"86.000","renk":"Siyah"},{"konum":"Esenyurt","model":"A3 Hatchback 1.6","model_yil":"2004","km"